In [34]:
import re
from IPython.display import display as dd

In [58]:
file = open('./schema.prisma').read()

# remove comments
file = re.sub(r'//(.)*', ' ', file)

ret = ''
# remove '@' tokens
save = 0
for i in range(len(file)):
    if file[i] == '@':
        if i - save:
            ret += file[save:i]
        i += 1
        parcount = 0
        while file[i] not in  [' ', '\n'] or parcount > 0:
            if file[i] == '(':
                parcount += 1
            if file[i] == ')':
                parcount -= 1
            i+=1
        save = i
ret += file[save:]
file = ret

file = file.replace(' Int ', ' Number ')
file = file.replace('DateTime', 'String')
file = file.replace('?', '')
file = file.replace('\n', ' ')
file = file.replace('\t', '')
file = re.sub('\\ +', ' ', file)
tokens = file.split(' ')

for i, tok in enumerate(tokens):
    if tok == 'model':
        tokens = tokens[i:]
        break

models = []
i = 0
while i < len(tokens):
    if tokens[i] == 'enum':
        break
    if tokens[i] == 'model' and tokens[i + 2] == '{':
        i += 1 # skip 'model'
        model = { 'name': tokens[i], 'vars': [] }
        i += 2 # skip model name + '{'
        j = i
        while tokens[j] != '}':
            j+=1
        model['vars'] = [ c for c in zip(tokens[i:j:2], tokens[i+1:j:2]) ]
    elif tokens[i] == '}':
        models.append(model)
    i+=1
    

In [59]:
for m in models:
    print(m['name'])

User
Follows
Blocks
Message
Channel
Subscription
Game
Avatar


In [61]:
tsmodels = ''
# dd(models[0])
for model in models:
    tsmodels += '/*\n'
    tsmodels += '** %s\n' % model['name']
    tsmodels += '*/\n'
    tsmodels += 'export class %s implements I%s {' % (model['name'],model['name'])
    for var in model['vars']:
        tsmodels += '\n  %-30s: %s;' % (var[0], var[1])
    tsmodels += '\n\n  constructor('
    for var in model['vars']:
        tsmodels += '\n    %-30s: %s,' % (var[0], var[1])
    tsmodels += '\n  ) {'
    for var in model['vars']:
        tsmodels += '\n    this.%-30s = %s;' % (var[0], var[0])
    tsmodels += '\n  }\n}\ninterface I%s {' % (model['name'])
    for var in model['vars']:
        tsmodels += '\n  %-30s: %s;' % (var[0], var[1])
    tsmodels += '\n}\n\n'
    
print(tsmodels)

/*
** User
*/
export class User implements IUser {
  username                      : String;
  email                         : String;
  createdAt                     : String;
  updatedAt                     : String;
  TwoFA                         : Boolean;
  password                      : String;
  salt                          : String;
  identification_token          : String;
  refresh_token                 : String;
  channelSubscriptions          : Subscription[];
  messages                      : Message[];
  gameHistoryPOne               : Game[];
  gameHistoryPTwo               : Game[];
  followedBy                    : Follows[];
  following                     : Follows[];
  blockedBy                     : Blocks[];
  blocking                      : Blocks[];
  avatars                       : Avatar;
  victoriesAsPOne               : Number;
  victoriesAsPTwo               : Number;
  defeatsAsPOne                 : Number;
  defeatsAsPTwo                 : Number;

  